<a href="https://colab.research.google.com/github/looee1q/PracticalML/blob/5.4-logictic-regression/LogReg_final_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [44]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [45]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [46]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [47]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [48]:
data.describe(include="object")

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [49]:
data = data.drop(columns=["Education", "Occupation", "Commute Distance", "Region"])

Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [50]:
data_bi = pd.get_dummies(data=data, drop_first=False, dtype=int)
for column in data_bi.columns:
  if (data_bi[column].isin([0, 1]).all()):
    if (data_bi[column].sum() < 500):
      data_bi = data_bi.drop(columns=[column])
data_bi.describe()

,ID,Income,Children,Cars,Age,Marital Status_Married,Gender_Male,Home Owner_Yes,Purchased Bike_No
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,19965.992000,56140.000000,1.908000,1.452000,44.190000,0.539000,0.509000,0.685000,0.519000
std,5347.333948,31081.609779,1.626094,1.124705,11.353537,0.498726,0.500169,0.464748,0.499889
min,11000.000000,10000.000000,0.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000
25%,15290.750000,30000.000000,0.000000,1.000000,35.000000,0.000000,0.000000,0.000000,0.000000
50%,19744.000000,60000.000000,2.000000,1.000000,43.000000,1.000000,1.000000,1.000000,1.000000
75%,24470.750000,70000.000000,3.000000,2.000000,52.000000,1.000000,1.000000,1.000000,1.000000
max,29447.000000,170000.000000,5.000000,4.000000,89.000000,1.000000,1.000000,1.000000,1.000000


Удалите столбец `ID`, так как он по сути нечисловой.

In [51]:
data_bi = data_bi.drop(columns=['ID'])

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [52]:
X = data_bi.drop(columns=['Purchased Bike_No'])
y = data_bi["Purchased Bike_No"]

X, y

(     Income  Children  Cars  Age  Marital Status_Married  Gender_Male  \
 0     40000         1     0   42                       1            0   
 1     30000         3     1   43                       1            1   
 2     80000         5     2   60                       1            1   
 3     70000         0     1   41                       0            1   
 4     30000         0     0   36                       0            1   
 ..      ...       ...   ...  ...                     ...          ...   
 995   60000         2     2   54                       1            1   
 996   70000         4     0   35                       0            1   
 997   60000         2     0   38                       1            1   
 998  100000         3     3   38                       0            1   
 999   60000         3     2   53                       0            1   
 
      Home Owner_Yes  
 0                 1  
 1                 1  
 2                 0  
 3                

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [53]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [54]:
X_train.shape[0]

700

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [55]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
min_max_scaler.fit(X_train)
X_train = pd.DataFrame(min_max_scaler.transform(X_train))
X_test = pd.DataFrame(min_max_scaler.transform(X_test))
X_train.columns = X.columns
X_test.columns = X.columns

Теперь обучите логистическую регрессию на тренировочных данных

In [56]:
from sklearn.linear_model import LogisticRegression

log_regession = LogisticRegression()
log_regession.fit(X_train, y_train)

LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [57]:
y_prediction = log_regession.predict(X_test)

Оцените значение accuracy на трейне и на тесте.

In [58]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_prediction)
accuracy

0.5766666666666667

Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [59]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures()
poly_features.fit(X_train)
X_train_poly = pd.DataFrame(poly_features.transform(X_train))
X_test_poly = pd.DataFrame(poly_features.transform(X_test))

**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [60]:
log_regression_poly = LogisticRegression()
log_regression_poly.fit(X_train_poly, y_train)
y_pred_poly = log_regression_poly.predict(X_test_poly)
accuracy_poly = accuracy_score(y_test, y_pred_poly)
accuracy_poly

0.6233333333333333

**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [61]:
from sklearn.metrics import recall_score

recall = recall_score(y_test, y_pred_poly)
accuracy_poly = accuracy_score(y_test, y_pred_poly)
recall, accuracy_poly

(0.6959459459459459, 0.6233333333333333)

Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [62]:
X_trainS, X_val, y_trainS, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

X_trainS.shape, X_val.shape

((490, 7), (210, 7))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [63]:
BestThr = -1
RecMax = -1
BestAcc = -1

log_regression_new = LogisticRegression()
log_regression_new.fit(X_trainS, y_trainS)
probs_new = log_regression_new.predict_proba(X_val)[:, 1]

for thr in np.arange(0, 1, 0.01):
  y_prediction_thr = (probs_new >= thr).astype(int)
  accuracy_thr = accuracy_score(y_val, y_prediction_thr)
  if (accuracy_thr >= 0.6):
    recall = recall_score(y_val, y_prediction_thr)
    if (RecMax < recall):
      RecMax = recall
      BestThr = thr
      BestAcc = accuracy_thr

print(BestThr, RecMax, BestAcc)

0.43 0.8240740740740741 0.6


Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [64]:
log_regession_end = LogisticRegression()
log_regession_end.fit(X_train, y_train)
probs_end = log_regession_end.predict_proba(X_test)[:, 1]
y_prediction_end = (probs_end >= BestThr).astype(int)
recall_end = recall_score(y_test, y_prediction_end)
recall_end

0.8108108108108109

**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.